# Introduction

This notebook presents a benchmark comparison between `power-grid-model`, 
[`pandapower`](http://www.pandapower.org/), and [`OpenDSS`](https://www.epri.com/pages/sa/opendss).
It runs several calculations, measures the calculation time, and compares the results.

## Test Network

The test network is fictionally generated using pre-defined random criteria. It is a radial network as follows:

```

source --- source_node ---| ---line--- node ---line--- node ...   (n_node_per_feeder)
                          |              |              |
                          |            load            load ...
                          |
                          | ---line--- ...
                          | .
                          | .
                          | .
                          | (n_feeder)

```

There is a node which is connected to a source (external network). From the source node there are `n_feeder` feeders. For each feeder there are `n_node_per_feeder` nodes, lines, and asymmetric loads. There are in total `n_feeder * n_node_per_feeder + 1` nodes in the network.


## Calculation

The notebook runs a power flow calculation with the same input data in `power-grid-model`, `pandapower`, and `OpenDSS`. It runs the following calculations:

* Single calculation with solver initialization.
* Single calculation without solver initialization (using pre-cached internal matrices).
* Time-series calculation
* N-1 calculation (not for `OpenDSS`)

The calculation is run symmetrically (not for `OpenDSS`) and asymmetrically. Both `power-grid-model` and `pandapower` supports asymmetric loads in symmetric calculations: the three-phase load is aggregated into one symmetric load. In `OpenDSS` we only run asymmetric calculations 

We use the Newton-Raphson method for `power-grid-model` and `pandapower`. In addition, we also use the iterative current and linear method in `power-grid-model` to see how much performance you can gain in exchange for accuracy. For `OpenDSS` the default fix point method is used.

## Results Comparison

The following results are compared between `power-grid-model`, `pandapower`, and `OpenDSS`:

* Per unit voltage of nodes (buses). For asymmetric calculation, it compares the value per phase.
* Loading of the lines. (not for `OpenDSS`)

It only compares the results of the Newton-Raphson in `pandapower` and `power-grid-model`, as well as the fix point method in `OpenDSS`, since the linear method of `power-grid-model` will produce a less accurate result.

## Performance Benchmark

The CPU time is measured for the **calculation part** of the program. The data preparation and model initialization is not measured. Furthermore, the single calculation is benchmarked with and without solver initialization. The former needs to execute connectivity check and initialize internal matrices (for example node admittance matrix). The latter uses the pre-cached connectivity and internal matrices.

# Preparation

## Import Libraries

We import neede libraries here. The fictional network generation and time-series profile generation is in a different Python file
[generate_fictional_dataset.py](./generate_fictional_dataset.py).

In [1]:
import time
from pathlib import Path
from copy import deepcopy
import numba

import numpy as np
import pandapower as pp
import pandas as pd
import power_grid_model as pgm

from generate_fictional_dataset import generate_fictional_grid

from dss import DSS as dss_engine

import warnings
warnings.filterwarnings('ignore')




## Prepare Tables

The performance comparison table and result deviation table is initialized below.

In [2]:
# summary
summary_df = pd.DataFrame(
    np.full(shape=(8, 6), dtype=np.float64, fill_value=np.inf),
    columns=['PGM Linear Impedance', 'PGM Linear Current' ,'PGM Newton-Raphson', 'PGM Iterative Current', 'PandaPower Newton-Raphson', 'OpenDSS Fix Point'],
    index=[
        'Symmetric calculation with solver initialization',
        'Symmetric calculation without solver initialization',
        'Asymmetric calculation with solver initialization',
        'Asymmetric calculation without solver initialization',
        'Time series symmetric calculation',
        'Time series asymmetric calculation',
        'N-1 symmetric calculation',
        'N-1 asymmetric calculation',
    ]
)

comparison_df = pd.DataFrame(
    np.full(shape=(6, 3), dtype=np.float64, fill_value=np.nan),
    columns=['Deviation Voltage (p.u.)', 'Deviation Loading (p.u.)', 'Deviation Voltage (p.u.) OpenDSS'],
    index=[
        'Symmetric calculation',
        'Asymmetric calculation',
        'Time series symmetric calculation',
        'Time series asymmetric calculation',
        'N-1 symmetric calculation',
        'N-1 asymmetric calculation',
    ]
)

## Simulation Parameters

The simulation parameters, for example, the total number of feeders `n_feeder`, are defined below.

In [3]:
# fictional grid parameters

n_node_per_feeder = 10
n_feeder = 100

cable_length_km_min = 0.8
cable_length_km_max = 1.2
load_p_w_max = 0.4e6 * 0.8
load_p_w_min = 0.4e6 * 1.2
pf = 0.95

load_scaling_min = 0.5
load_scaling_max = 1.5
n_step = 1000


use_lightsim2grid = True

In [4]:
# override small network

n_node_per_feeder = 3
n_feeder = 2
n_step = 10


In [5]:
# derived values

n_node = n_node_per_feeder * n_feeder + 1
n_line = n_node_per_feeder * n_feeder
n_load = n_node_per_feeder * n_feeder

## Pre-cache Library

To make a fair comparison, we run one small network so that `pandapower` can cache their dependent libraries into the memory.

* For `pandapower` the `numba` functions are JIT compiled and cached in the memory.

In [6]:
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=3,
    n_feeder=2,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf,
    n_step=n_step,
    load_scaling_min=load_scaling_min,
    load_scaling_max=load_scaling_max,
)

pp.runpp(fictional_dataset['pp_net'], algorithm='nr', calculate_voltage_angles=True, distributed_slack=True, 
         lightsim2grid=use_lightsim2grid)
pgm_model = pgm.PowerGridModel(fictional_dataset['pgm_dataset'])
pgm_result = pgm_model.calculate_power_flow()

In [7]:
from power_grid_model.utils import json_serialize_to_file

In [8]:
fictional_dataset['pgm_dataset']

{'node': array([(0, 10000.), (1, 10000.), (2, 10000.), (3, 10000.), (4, 10000.),
        (5, 10000.), (6, 10000.)],
       dtype={'names': ['id', 'u_rated'], 'formats': ['<i4', '<f8'], 'offsets': [0, 8], 'itemsize': 16, 'aligned': True}),
 'line': array([( 7, 0, 1, 1, 1, 0.0642301 , 0.10501112, 4.07810161e-07, 0., 0.15904596, 0.10195254, 6.72886765e-07, 0., 1000.),
        ( 8, 1, 2, 1, 1, 0.06842277, 0.1118658 , 4.34430299e-07, 0., 0.16942782, 0.10860757, 7.16809993e-07, 0., 1000.),
        ( 9, 2, 3, 1, 1, 0.06558964, 0.10723385, 4.16442140e-07, 0., 0.16241243, 0.10411054, 6.87129531e-07, 0., 1000.),
        (10, 0, 4, 1, 1, 0.06413106, 0.10484919, 4.07181309e-07, 0., 0.15880071, 0.10179533, 6.71849160e-07, 0., 1000.),
        (11, 4, 5, 1, 1, 0.0610761 , 0.09985458, 3.87784768e-07, 0., 0.15123606, 0.09694619, 6.39844867e-07, 0., 1000.),
        (12, 5, 6, 1, 1, 0.06667653, 0.10901084, 4.23343058e-07, 0., 0.16510379, 0.10583576, 6.98516046e-07, 0., 1000.)],
       dtype={'names': ['i

# Generate Dataset

First generate the fictional datasets.

In [9]:
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=n_node_per_feeder,
    n_feeder=n_feeder,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf,
    n_step=n_step,
    load_scaling_min=load_scaling_min,
    load_scaling_max=load_scaling_max,
)

pp_net = deepcopy(fictional_dataset["pp_net"])
pgm_dataset = fictional_dataset["pgm_dataset"]
pgm_update_dataset = fictional_dataset["pgm_update_dataset"]
pp_time_series_dataset = fictional_dataset["pp_time_series_dataset"]



In [10]:
# initialize DSS

dss_engine.ClearAll()
dss_engine.Text.Command = f"compile {fictional_dataset['dss_file']}"


# Benchmark Function

In [11]:
def benchmark_pgm_power_flow(symmetric: bool, calculation_type: str, update_data=None, with_intialization: bool = False):
    method_dict = {
        'linear': 'PGM Linear Impedance',
        'linear_current': 'PGM Linear Current',
        'iterative_current': 'PGM Iterative Current',
        'newton_raphson': 'PGM Newton-Raphson'
    }
    for method in ['linear', 'linear_current', 'iterative_current', 'newton_raphson']:
        model_instance = pgm.PowerGridModel(pgm_dataset)
        # cache internal state if we do not benchmark solver intialization
        if not with_intialization:
            model_instance.calculate_power_flow(symmetric=symmetric, calculation_method=method)
        start = time.time()
        pgm_result = model_instance.calculate_power_flow(
            symmetric=symmetric, 
            calculation_method=method, 
            update_data=update_data, 
            output_component_types=['node', 'line'])
        end = time.time()
        summary_df.loc[calculation_type, method_dict[method]] = end - start
    return pgm_result

# Single Calculation

We begin with a single power flow calculation.

## Symmetric

### power-grid-model

In [12]:
benchmark_pgm_power_flow(symmetric=True, calculation_type='Symmetric calculation with solver initialization', with_intialization=True)
pgm_result = benchmark_pgm_power_flow(symmetric=True, calculation_type='Symmetric calculation without solver initialization', with_intialization=False)


### Newton-Raphson Method of pandapower

In [13]:
# first calculation with solver initialization
start = time.time()
pp.runpp(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True, lightsim2grid=use_lightsim2grid)
end = time.time()
summary_df.loc['Symmetric calculation with solver initialization', 'PandaPower Newton-Raphson'] = end - start

# second calculation with existing solver
start = time.time()
pp.runpp(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True, lightsim2grid=use_lightsim2grid)
end = time.time()
summary_df.loc['Symmetric calculation without solver initialization', 'PandaPower Newton-Raphson'] = end - start

### Calculate Deviation for Newton-Raphson Method

In [14]:
comparison_df.loc['Symmetric calculation', 'Deviation Voltage (p.u.)'] = \
    np.abs(pp_net.res_bus['vm_pu'] - pgm_result['node']['u_pu']).max()
comparison_df.loc['Symmetric calculation', 'Deviation Loading (p.u.)'] = \
    np.abs(pp_net.res_line['loading_percent'] * 1e-2 - pgm_result['line']['loading']).max()

## Asymmetric

### power-grid-model

In [15]:
benchmark_pgm_power_flow(symmetric=False, calculation_type='Asymmetric calculation with solver initialization', with_intialization=True)
pgm_result = benchmark_pgm_power_flow(symmetric=False, calculation_type='Asymmetric calculation without solver initialization', with_intialization=False)

### Newton-Raphson Method of pandapower

In [16]:
# first calculation with solver initialization
start = time.time()
pp.runpp_3ph(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True, lightsim2grid=use_lightsim2grid)
end = time.time()
summary_df.loc['Asymmetric calculation with solver initialization', 'PandaPower Newton-Raphson'] = end - start

# second calculation with existing solver
start = time.time()
pp.runpp_3ph(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True, lightsim2grid=use_lightsim2grid)
end = time.time()
summary_df.loc['Asymmetric calculation without solver initialization', 'PandaPower Newton-Raphson'] = end - start


### Fix Point Method of OpenDSS

In [17]:
# first calculation with solver initialization
start = time.time()
dss_engine.Text.Command = "set mode=snapshot"
dss_engine.Text.Command = "set controlmode=static"
dss_engine.ActiveCircuit.Solution.Solve()
end = time.time()
summary_df.loc['Asymmetric calculation with solver initialization', 'OpenDSS Fix Point'] = end - start

# second calculation with existing solver
start = time.time()
dss_engine.ActiveCircuit.Solution.Solve()
end = time.time()
summary_df.loc['Asymmetric calculation without solver initialization', 'OpenDSS Fix Point'] = end - start


### Calculate Deviation for Newton-Raphson Method

In [18]:
comparison_df.loc['Asymmetric calculation', 'Deviation Voltage (p.u.)'] = \
    np.abs(pp_net.res_bus_3ph[['vm_a_pu', 'vm_b_pu', 'vm_c_pu']].to_numpy() - pgm_result['node']['u_pu']).max()
comparison_df.loc['Asymmetric calculation', 'Deviation Loading (p.u.)'] = \
    np.abs(pp_net.res_line_3ph['loading_percent'] * 1e-2 - pgm_result['line']['loading']).max()
comparison_df.loc['Asymmetric calculation', 'Deviation Voltage (p.u.) OpenDSS'] = \
    np.abs(dss_engine.ActiveCircuit.AllBusVmagPu.reshape(-1, 3) - pgm_result['node']['u_pu']).max()

# Time Series Calculation

We execute a time-series power flow with `n_step` timestamps. 

## Preparation

The load profile is randomly generated by the function `generate_time_series`. It produces the relevant input format for both libraries.

In [19]:

time_steps = np.arange(n_step)

for x, y in zip(['p', 'q'], ['mw', 'mvar']):
    for p in ['a', 'b', 'c']:
        name = f'{x}_{p}_{y}'
        pp.control.ConstControl(
            pp_net,
            element='asymmetric_load',
            element_index=pp_net.asymmetric_load.index,
            variable=name,
            data_source=pp_time_series_dataset[name],
            profile_name=pp_net.asymmetric_load.index
        )


## Symmetric

### power-grid-model

In [20]:
pgm_result = benchmark_pgm_power_flow(symmetric=True, calculation_type='Time series symmetric calculation', update_data=pgm_update_dataset)



### Newton-Raphson Method of pandapower

In [21]:
pp.timeseries.OutputWriter(
    pp_net,
    log_variables=[
        ('res_bus', 'vm_pu'),
        ('res_line', 'loading_percent'),
    ]
)

start = time.time()
pp.timeseries.run_timeseries(
    pp_net, run=pp.runpp, time_steps=time_steps,
    calculate_voltage_angles=True, distributed_slack=True, lightsim2grid=use_lightsim2grid
)
end = time.time()
summary_df.loc['Time series symmetric calculation', 'PandaPower Newton-Raphson'] = end - start

100%|██████████| 10/10 [00:00<00:00, 103.65it/s]


### Calculate Deviation for Newton-Raphson Method

In [22]:
pp_u_pu = pp_net.output_writer.iloc[0, 0].output['res_bus.vm_pu'].to_numpy()
comparison_df.loc['Time series symmetric calculation', 'Deviation Voltage (p.u.)'] = \
    np.abs(pp_u_pu - pgm_result['node']['u_pu']).max()
pp_loading = pp_net.output_writer.iloc[0, 0].output['res_line.loading_percent'].to_numpy() * 1e-2
comparison_df.loc['Time series symmetric calculation', 'Deviation Loading (p.u.)'] = \
    np.abs(pp_loading - pgm_result['line']['loading']).max()


## Asymmetric

### power-grid-model

In [23]:
pgm_result = benchmark_pgm_power_flow(symmetric=False, calculation_type='Time series asymmetric calculation', update_data=pgm_update_dataset)


### Newton-Raphson Method of pandapower

In [24]:
del pp_net.output_writer
ow = pp.timeseries.OutputWriter(pp_net)
ow.log_variable('res_bus_3ph', 'vm_a_pu', index=pp_net.bus.index)
ow.log_variable('res_bus_3ph', 'vm_b_pu', index=pp_net.bus.index)
ow.log_variable('res_bus_3ph', 'vm_c_pu', index=pp_net.bus.index)
ow.log_variable('res_line_3ph', 'loading_percent', index=pp_net.line.index)

# run
start = time.time()
pp.timeseries.run_timeseries(
    pp_net,
    run=pp.runpp_3ph,
    time_steps=time_steps,
    calculate_voltage_angles=True,
    distributed_slack=True, 
    lightsim2grid=use_lightsim2grid
)
end = time.time()
summary_df.loc['Time series asymmetric calculation', 'PandaPower Newton-Raphson'] = end - start

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 56.12it/s]


### Fix Point Method of OpenDSS

In [25]:
# first calculation with solver initialization
start = time.time()
dss_engine.Text.Command = "set mode=Daily"
dss_engine.Text.Command = "set Stepsize=3600s"
dss_engine.Text.Command = f"set Number={n_step}"
dss_engine.Text.Command = "set controlmode=static"
dss_engine.ActiveCircuit.Solution.Solve()
end = time.time()
summary_df.loc['Time series asymmetric calculation', 'OpenDSS Fix Point'] = end - start


In [26]:
# get results
all_results = []

# source
flag = dss_engine.ActiveCircuit.Monitors.First
assert flag != 0
all_results += [dss_engine.ActiveCircuit.Monitors.Channel(1), dss_engine.ActiveCircuit.Monitors.Channel(2), dss_engine.ActiveCircuit.Monitors.Channel(3)]

# others
flag = dss_engine.ActiveCircuit.Monitors.Next
while flag != 0:
    all_results.append(dss_engine.ActiveCircuit.Monitors.Channel(1))
    flag = dss_engine.ActiveCircuit.Monitors.Next

dss_voltage = np.stack(all_results, axis=1)
dss_voltage = dss_voltage.reshape(n_step, -1, 3)
dss_voltage /= 10e3 / np.sqrt(3)


### Calculate Deviation for Newton-Raphson Method

In [27]:
pp_u_pu = []
for p in ['a', 'b', 'c']:
    pp_u_pu.append(pp_net.output_writer.iloc[0, 0].output[f'res_bus_3ph.vm_{p}_pu'].to_numpy())
pp_u_pu = np.stack(pp_u_pu, axis=-1)
comparison_df.loc['Time series asymmetric calculation', 'Deviation Voltage (p.u.)'] = \
    np.abs(pp_u_pu - pgm_result['node']['u_pu']).max()

pp_loading = pp_net.output_writer.iloc[0, 0].output[r'res_line_3ph.loading_percent'].to_numpy() * 1e-2
comparison_df.loc['Time series asymmetric calculation', 'Deviation Loading (p.u.)'] = \
    np.abs(pp_loading - pgm_result['line']['loading']).max()

comparison_df.loc['Time series asymmetric calculation', 'Deviation Voltage (p.u.) OpenDSS'] = \
    np.abs(dss_voltage - pgm_result['node']['u_pu']).max()

# N-1 Scenario Calculation

We execute a N-1 scenario calculation. There are `n_line` scenarios. In each scenario one line will be disabled. Since the original network is radial, part of the network will be unenergized due to the switch-off of a line.

## Preparation

The N-1 scenario input is generated for `power-grid-model`. Since `pandapower` does not have a built-in N-1 scenario calculation, we execute the power flow per scenario in a loop.

In [28]:
# re-generate dataset

pp_net = deepcopy(fictional_dataset["pp_net"])
pgm_dataset = fictional_dataset["pgm_dataset"]

# update dataset for power grid model
# disable one line per batch
pgm_line_profile = pgm.initialize_array("update", "line", (n_line, n_line))
pgm_line_profile["id"] = pgm_dataset["line"]["id"]
pgm_line_profile["from_status"] = 1
pgm_line_profile["to_status"] = 1
np.fill_diagonal(pgm_line_profile["from_status"], 0)
np.fill_diagonal(pgm_line_profile["to_status"], 0)
pgm_update_dataset = {"line": pgm_line_profile}

## Symmetric

### power-grid-model

In [29]:
pgm_result = benchmark_pgm_power_flow(symmetric=True, calculation_type='N-1 symmetric calculation', update_data=pgm_update_dataset)


### Newton-Raphson Method of pandapower

In [30]:
# prepare pandapower result dataset
pp_u_pu = np.empty(shape=(n_line, n_node), dtype=np.float64)
pp_loading = np.empty(shape=(n_line, n_line), dtype=np.float64)

start = time.time()
# loop to calcualte pandapower N-1
for i in pp_net.line.index:
    # set one line out of service
    pp_net.line.loc[i, 'in_service'] = False
    pp.runpp(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True, lightsim2grid=use_lightsim2grid)
    # restore that line
    pp_net.line.loc[i, 'in_service'] = True
    # get result
    pp_u_pu[i, :] = pp_net.res_bus['vm_pu']
    pp_loading[i, :] = pp_net.res_line['loading_percent'] * 1e-2
end = time.time()
summary_df.loc['N-1 symmetric calculation', 'PandaPower Newton-Raphson'] = end - start

# set nan to 0.0 to make a meaningful comparison
pp_u_pu[np.isnan(pp_u_pu)] = 0.0
pp_loading[np.isnan(pp_loading)] = 0.0

### Calculate Deviation for Newton-Raphson Method

In [31]:
comparison_df.loc['N-1 symmetric calculation', 'Deviation Voltage (p.u.)'] = \
    np.abs(pp_u_pu - pgm_result['node']['u_pu']).max()
comparison_df.loc['N-1 symmetric calculation', 'Deviation Loading (p.u.)'] = \
    np.abs(pp_loading - pgm_result['line']['loading']).max()

## Asymmetric

### power-grid-model

In [32]:
pgm_result = benchmark_pgm_power_flow(symmetric=False, calculation_type='N-1 asymmetric calculation', update_data=pgm_update_dataset)


### Newton-Raphson Method of pandapower

In [33]:
%%capture

# prepare pandapower result dataset
pp_u_pu = np.empty(shape=(n_line, n_node, 3), dtype=np.float64)
pp_loading = np.empty(shape=(n_line, n_line), dtype=np.float64)

start = time.time()
# loop to calcualte pandapower N-1
for i in pp_net.line.index:
    # set one line out of service
    pp_net.line.loc[i, 'in_service'] = False
    pp.runpp_3ph(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True, lightsim2grid=use_lightsim2grid)
    # restore that line
    pp_net.line.loc[i, 'in_service'] = True
    # get result
    pp_u_pu[i, ...] = pp_net.res_bus_3ph[['vm_a_pu', 'vm_b_pu', 'vm_c_pu']]
    pp_loading[i, :] = pp_net.res_line_3ph['loading_percent'] * 1e-2
end = time.time()
summary_df.loc['N-1 asymmetric calculation', 'PandaPower Newton-Raphson'] = end - start

# set nan to 0.0 to make a meaningful comparison
pp_u_pu[np.isnan(pp_u_pu)] = 0.0
pp_loading[np.isnan(pp_loading)] = 0.0

### Calculate Deviation for Newton-Raphson Method

In [34]:
comparison_df.loc['N-1 asymmetric calculation', 'Deviation Voltage (p.u.)'] = \
    np.abs(pp_u_pu - pgm_result['node']['u_pu']).max()
comparison_df.loc['N-1 asymmetric calculation', 'Deviation Loading (p.u.)'] = \
    np.abs(pp_loading - pgm_result['line']['loading']).max()

# Summary

## Deviation of Results

Below is the table of deviation between the results from `power-grid-model`, `pandapower`, and `OpenDSS`. It matches to the order of `1e-6`. Note there are no comparisons for `OpenDSS` for symmetric calculations.

In [35]:
display(comparison_df)

,Deviation Voltage (p.u.),Deviation Loading (p.u.),Deviation Voltage (p.u.) OpenDSS
Symmetric calculation,1.517786e-11,2.034727e-10,NaN
Asymmetric calculation,4.070078e-13,4.927142e-12,2.255973e-13
Time series symmetric calculation,2.968603e-11,3.913039e-10,NaN
Time series asymmetric calculation,2.374989e-12,3.728935e-11,1.484683e-07
N-1 symmetric calculation,1.517719e-11,2.034866e-10,NaN
N-1 asymmetric calculation,4.123368e-13,4.930986e-12,NaN


## Performance Comparison

Below is the table of the measured time of all calculations. Note there are no comparisons for `OpenDSS` for symmetric calculations.

In [36]:
display(summary_df)

,PGM Linear Impedance,PGM Linear Current,PGM Newton-Raphson,PGM Iterative Current,PandaPower Newton-Raphson,OpenDSS Fix Point
Symmetric calculation with solver initialization,0.000458,0.000981,0.000092,0.000221,0.014860,inf
Symmetric calculation without solver initialization,0.000053,0.000054,0.000054,0.000056,0.005293,inf
Asymmetric calculation with solver initialization,0.000749,0.000160,0.000096,0.000098,0.393961,0.000759
Asymmetric calculation without solver initialization,0.000055,0.000057,0.000065,0.000057,0.013605,0.000156
Time series symmetric calculation,0.001293,0.000187,0.000182,0.000181,0.135885,inf
Time series asymmetric calculation,0.000546,0.000410,0.000831,0.000694,0.181544,0.000902
N-1 symmetric calculation,0.000469,0.000417,0.001040,0.000567,0.077501,inf
N-1 asymmetric calculation,0.001124,0.000336,0.000258,0.000237,0.103225,inf


In [37]:
relative_df = summary_df.div(summary_df['PandaPower Newton-Raphson'], axis=0)
speedup_df = 1 / relative_df
display(speedup_df.style.format("{:0.2f}x"))

,PGM Linear Impedance,PGM Linear Current,PGM Newton-Raphson,PGM Iterative Current,PandaPower Newton-Raphson,OpenDSS Fix Point
Symmetric calculation with solver initialization,32.45x,15.15x,161.47x,67.24x,1.00x,0.00x
Symmetric calculation without solver initialization,99.55x,98.23x,98.23x,94.46x,1.00x,0.00x
Asymmetric calculation with solver initialization,526.07x,2462.58x,4100.23x,4020.42x,1.00x,519.13x
Asymmetric calculation without solver initialization,247.02x,239.76x,209.02x,237.76x,1.00x,87.25x
Time series symmetric calculation,105.10x,726.04x,746.00x,750.91x,1.00x,0.00x
Time series asymmetric calculation,332.37x,442.96x,218.49x,261.67x,1.00x,201.34x
N-1 symmetric calculation,165.26x,185.86x,74.52x,136.64x,1.00x,0.00x
N-1 asymmetric calculation,91.83x,307.06x,400.15x,435.13x,1.00x,0.00x
